In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key

tavily_api_key = os.getenv("TAVILY_API_KEY")
os.environ["TAVILY_API_KEY"] = tavily_api_key

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator, sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

In [3]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [4]:
def node1(state: AgentState):
    print(f"node1, count: {state['count']}")
    return {
        "lnode": "node_1",
        "count": 1,
    }

In [5]:
def node2(state: AgentState):
    print(f"node2, count: {state['count']}")
    return {
        "lnode": "node_2",
        "count": 1,
    }

The graph goes N1->N2->N1... but breaks after count reaches 3.

In [6]:
def should_continue(state):
    return state["count"] < 3

In [7]:
builder = StateGraph(AgentState)
builder.add_node("Node1",node1)
builder.add_node("Node2",node2)

builder.add_edge("Node1", "Node2")
builder.add_conditional_edges(
    "Node2",
    should_continue,
    {True: "Node1", False: END}
)
builder.set_entry_point("Node1")

In [10]:
conn = sqlite3.Connection("checkpoints.sqlite", check_same_thread=False)
memory = SqliteSaver(conn)
graph = builder.compile(checkpointer=memory)

In [11]:
thread = {"configurable": {"thread_id": str(1)}}
graph.invoke({"count":0, "scratch":"hi"},thread)

node1, count: 0
node2, count: 1
node1, count: 2
node2, count: 3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [12]:
graph.get_state(thread)

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c559-6431-8009-2091696b4f1c'}}, metadata={'source': 'loop', 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}, 'step': 9, 'parents': {}, 'thread_id': '1'}, created_at='2025-04-03T14:53:10.357304+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c539-69c6-8008-c3024f5dd25e'}}, tasks=())

In [13]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c559-6431-8009-2091696b4f1c'}}, metadata={'source': 'loop', 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}, 'step': 9, 'parents': {}, 'thread_id': '1'}, created_at='2025-04-03T14:53:10.357304+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c539-69c6-8008-c3024f5dd25e'}}, tasks=()) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c539-69c6-8008-c3024f5dd25e'}}, metadata={'source': 'loop', 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}, 'step': 8, 'parents': {}, 'thread_id': '1'}, created_at='2025-04-03T14:53:10.344339+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b

In [14]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c559-6431-8009-2091696b4f1c'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c539-69c6-8008-c3024f5dd25e'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c534-6bd5-8007-4e546e2c7174'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c51c-662a-8006-ed450a05a470'}} 1
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c4d0-6e73-8005-7b99894bc844'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c4a7-6838-8004-355da06a0aae'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f010727-63e1-60b4-8003-6ba71c383f37'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f010727-63b2-6c77-8002-260ef229d4ea'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkp

In [15]:
states[-3]

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f010727-62ba-6e8c-8001-cf2c86ad76f1'}}

In [16]:
graph.get_state(states[-3])

StateSnapshot(values={'count': 0}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f010727-62ba-6e8c-8001-cf2c86ad76f1'}}, metadata={'source': 'loop', 'writes': {'llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Q0688Nzw1RiH63YOipy5r3ZA', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBexlP7Zjpr0FwmIJfp80utYu9sq', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8c2bcf4a-be6c-416c-a166-b4a867261b9

In [17]:
graph.invoke(None, states[-3])

{'count': 0}

In [18]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c559-6431-8009-2091696b4f1c'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c539-69c6-8008-c3024f5dd25e'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c534-6bd5-8007-4e546e2c7174'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c51c-662a-8006-ed450a05a470'}} 1
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c4d0-6e73-8005-7b99894bc844'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c4a7-6838-8004-355da06a0aae'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f010727-63e1-60b4-8003-6ba71c383f37'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f010727-63b2-6c77-8002-260ef229d4ea'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkp

In [19]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state,"\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c559-6431-8009-2091696b4f1c'}}, metadata={'source': 'loop', 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}, 'step': 9, 'parents': {}, 'thread_id': '1'}, created_at='2025-04-03T14:53:10.357304+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c539-69c6-8008-c3024f5dd25e'}}, tasks=()) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b5-c539-69c6-8008-c3024f5dd25e'}}, metadata={'source': 'loop', 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}, 'step': 8, 'parents': {}, 'thread_id': '1'}, created_at='2025-04-03T14:53:10.344339+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0109b

In [20]:
thread2 = {"configurable": {"thread_id": str(2)}}
graph.invoke({"count":0, "scratch":"hi"},thread2)

node1, count: 0
node2, count: 1
node1, count: 2
node2, count: 3


{'lnode': 'node_2', 'scratch': 'hi', 'count': 4}

In [21]:
states2 = []
for state in graph.get_state_history(thread2):
    states2.append(state.config)
    print(state.config, state.values['count'])   

{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109c6-7353-68cf-800c-c47313269dd3'}} 4
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109c6-7349-6cda-800b-82b8f4c8e3b0'}} 3
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109c6-7344-6f06-800a-c1c07a60b9b0'}} 2
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109c6-7331-68e3-8009-4b12058588af'}} 1
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109c6-732a-6264-8008-7a2b8cc2635a'}} 0
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109c6-7322-6d8c-8007-86ce5a11a9e6'}} 0
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f010734-fa65-65b2-8006-0352c05e683f'}} 0
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f010734-eeb6-6bf1-8005-90544b0e6a54'}} 0
{'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkp

In [23]:
save_state = graph.get_state(states2[-3])
save_state

StateSnapshot(values={'count': 0}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f010728-b12f-60dc-8001-3da87e6ee316'}}, metadata={'source': 'loop', 'writes': {'llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TXhEpd2BqYoM3e31G5PJlVuu', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBfXUu1bEJPWPnpmLX4KggCCfQw7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c953b961-10bd-4b48-bb16-170a692b0de9-

In [24]:
save_state.values["count"] = -3
save_state.values["scratch"] = "hello"
save_state

StateSnapshot(values={'count': -3, 'scratch': 'hello'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f010728-b12f-60dc-8001-3da87e6ee316'}}, metadata={'source': 'loop', 'writes': {'llm': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TXhEpd2BqYoM3e31G5PJlVuu', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 150, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_898ac29719', 'id': 'chatcmpl-BIBfXUu1bEJPWPnpmLX4KggCCfQw7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c953b961-10bd-4b

In [25]:
graph.update_state(thread2,save_state.values)

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0109ca-21f4-6a55-800d-090f0449484e'}}

In [26]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  #print latest 3
        break
    print(state, '\n')

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hello', 'count': 1}, next=('Node1',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109ca-21f4-6a55-800d-090f0449484e'}}, metadata={'source': 'update', 'writes': {'Node2': {'count': -3, 'scratch': 'hello'}}, 'step': 13, 'parents': {}, 'thread_id': '2'}, created_at='2025-04-03T15:02:16.938760+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109c6-7353-68cf-800c-c47313269dd3'}}, tasks=(PregelTask(id='e36261da-9006-cac1-d846-0b0e9d0704f8', name='Node1', path=('__pregel_pull', 'Node1'), error=None, interrupts=(), state=None, result=None),)) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109c6-7353-68cf-800c-c47313269dd3'}}, metadata={'source': 'loop', 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}, 'step': 12, 'parents': {}, 

In [28]:
graph.update_state(thread2,save_state.values, as_node="Node1")

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0109cc-c385-6af9-800f-1edc97bf2496'}}

In [29]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  #print latest 3
        break
    print(state, '\n')

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hello', 'count': -5}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109cc-c385-6af9-800f-1edc97bf2496'}}, metadata={'source': 'update', 'writes': {'Node1': {'count': -3, 'scratch': 'hello'}}, 'step': 15, 'parents': {}, 'thread_id': '2'}, created_at='2025-04-03T15:03:27.567333+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109cc-a15e-65f4-800e-4d4bfb4d05fa'}}, tasks=(PregelTask(id='1a59daf7-c87b-e70a-abc1-25a35a19d7ea', name='Node2', path=('__pregel_pull', 'Node2'), error=None, interrupts=(), state=None, result=None),)) 

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hello', 'count': -2}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109cc-a15e-65f4-800e-4d4bfb4d05fa'}}, metadata={'source': 'update', 'writes': {'Node1': {'count': -3, 'scratch': 'hello'}}, 'step': 14

In [30]:
graph.invoke(None,thread2)

node2, count: -5
node1, count: -4
node2, count: -3
node1, count: -2
node2, count: -1
node1, count: 0
node2, count: 1
node1, count: 2
node2, count: 3


{'lnode': 'node_2', 'scratch': 'hello', 'count': 4}

In [31]:
for state in graph.get_state_history(thread2):
    print(state,"\n")

StateSnapshot(values={'lnode': 'node_2', 'scratch': 'hello', 'count': 4}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109ce-6b78-6570-8018-434979b525b4'}}, metadata={'source': 'loop', 'writes': {'Node2': {'lnode': 'node_2', 'count': 1}}, 'step': 24, 'parents': {}, 'thread_id': '2'}, created_at='2025-04-03T15:04:12.021489+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109ce-6b75-6e7a-8017-9fa296007597'}}, tasks=()) 

StateSnapshot(values={'lnode': 'node_1', 'scratch': 'hello', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0109ce-6b75-6e7a-8017-9fa296007597'}}, metadata={'source': 'loop', 'writes': {'Node1': {'lnode': 'node_1', 'count': 1}}, 'step': 23, 'parents': {}, 'thread_id': '2'}, created_at='2025-04-03T15:04:12.020492+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': 